# Energy Consumption Predictions

Generate and visualize future energy consumption predictions.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../')
from src.data_processing import load_data, process_data
from src.model import EnergyConsumptionPredictor

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Load and process historical data
data = load_data('../data/energy_consumption.csv')
processed_data = process_data(data)

print(f"Historical data: {len(processed_data)} records")
print(f"Period: {processed_data['Date'].min()} to {processed_data['Date'].max()}")
processed_data.tail()

In [ ]:
# Train the model
predictor = EnergyConsumptionPredictor()
performance = predictor.train(processed_data)

print(f"Model trained: {performance['model_name']}")
print(f"R² Score: {performance['r2_score']:.3f}")
print(f"RMSE: {performance['rmse']:.2f} kWh")
print(f"MAE: {performance['mae']:.2f} kWh")

In [ ]:
# Generate 12-month predictions
predictions_df = predictor.predict_future(months=12)
print(f"Generated {len(predictions_df)} monthly predictions")
predictions_df.head()

In [ ]:
# Plot historical vs predictions
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Consumption
ax1.plot(processed_data['Date'], processed_data['Consumption'],
         marker='o', label='Historical', linewidth=2, alpha=0.8)
ax1.plot(predictions_df['Date'], predictions_df['Predicted_Consumption'],
         marker='s', label='Predicted', linewidth=2, alpha=0.8, color='orange')
ax1.set_title('Energy Consumption: Historical vs Predicted')
ax1.set_ylabel('Consumption (kWh)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Costs
ax2.plot(processed_data['Date'], processed_data['Cost'],
         marker='o', label='Historical', linewidth=2, alpha=0.8, color='green')
ax2.plot(predictions_df['Date'], predictions_df['Predicted_Cost'],
         marker='s', label='Predicted', linewidth=2, alpha=0.8, color='red')
ax2.set_title('Energy Costs: Historical vs Predicted')
ax2.set_ylabel('Cost (PLN)')
ax2.set_xlabel('Date')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Format x-axis
for ax in [ax1, ax2]:
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Seasonal analysis
predictions_df['Season'] = predictions_df['Month'].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Autumn', 10: 'Autumn', 11: 'Autumn'
})

seasonal_summary = predictions_df.groupby('Season').agg({
    'Predicted_Consumption': ['mean', 'sum'],
    'Predicted_Cost': ['mean', 'sum']
}).round(0)

print("Seasonal predictions summary:")
print(seasonal_summary)

In [ ]:
# Monthly breakdown visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Monthly consumption
months = ['Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May']
ax1.bar(months, predictions_df['Predicted_Consumption'], alpha=0.7, color='skyblue')
ax1.set_title('Predicted Monthly Consumption')
ax1.set_ylabel('Consumption (kWh)')
ax1.tick_params(axis='x', rotation=45)

# Seasonal comparison
seasonal_avg = predictions_df.groupby('Season')['Predicted_Consumption'].mean()
ax2.pie(seasonal_avg.values, labels=seasonal_avg.index, autopct='%1.1f%%', startangle=90)
ax2.set_title('Consumption by Season')

plt.tight_layout()
plt.show()

In [ ]:
# Cost analysis
total_predicted_cost = predictions_df['Predicted_Cost'].sum()
avg_monthly_cost = predictions_df['Predicted_Cost'].mean()
total_predicted_consumption = predictions_df['Predicted_Consumption'].sum()

print(f"12-month forecast summary:")
print(f"Total consumption: {total_predicted_consumption:.0f} kWh")
print(f"Total cost: {total_predicted_cost:.0f} PLN")
print(f"Average monthly cost: {avg_monthly_cost:.0f} PLN")
print(f"Cost per kWh: {total_predicted_cost/total_predicted_consumption:.2f} PLN")

# Compare with historical averages
hist_avg_monthly = processed_data['Consumption'].mean()
hist_avg_cost = processed_data['Cost'].mean()

print(f"\nComparison with historical averages:")
print(f"Consumption change: {((predictions_df['Predicted_Consumption'].mean() - hist_avg_monthly) / hist_avg_monthly * 100):+.1f}%")
print(f"Cost change: {((avg_monthly_cost - hist_avg_cost) / hist_avg_cost * 100):+.1f}%")

In [ ]:
# Export predictions
predictions_df.to_csv('../results/predictions_12_months.csv', index=False)
print("Predictions saved to ../results/predictions_12_months.csv")

# Summary table
summary_table = predictions_df[['Date', 'Predicted_Consumption', 'Predicted_Cost', 'Season']].copy()
summary_table['Date'] = summary_table['Date'].dt.strftime('%B %Y')
summary_table.columns = ['Month', 'Consumption (kWh)', 'Cost (PLN)', 'Season']
print("\nPredictions table:")
print(summary_table.to_string(index=False))